# До начала
[Данные здесь](https://cloud.mail.ru/public/26Da/AuAhoLbqZ)

Установите библиотеки MulticoreTSNE bokeh gensim

# Различные методы снижения размерностей
https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection, neighbors)
print(__doc__)

digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30

def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize=(15,15))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)


In [ ]:
n_img_per_row = 20
img = np.zeros((10 * n_img_per_row, 10 * n_img_per_row))
for i in range(n_img_per_row):
    ix = 10 * i + 1
    for j in range(n_img_per_row):
        iy = 10 * j + 1
        img[ix:ix + 8, iy:iy + 8] = X[i * n_img_per_row + j].reshape((8, 8))

plt.imshow(img, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.title('A selection from the 64-dimensional digits dataset');

In [ ]:
print("Computing random projection")
rp = random_projection.SparseRandomProjection(n_components=2, random_state=42)
X_projected = rp.fit_transform(X)
plot_embedding(X_projected, "Random Projection of the digits")

In [ ]:
print("Computing PCA projection")
t0 = time()
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(X)
plot_embedding(X_pca,
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0))


In [ ]:
print("Computing Isomap projection")
t0 = time()
X_iso = manifold.Isomap(n_neighbors, n_components=2).fit_transform(X)
print("Done.")
plot_embedding(X_iso,
               "Isomap projection of the digits (time %.2fs)" %
               (time() - t0))

In [ ]:
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
t0 = time()
X_tsne = tsne.fit_transform(X)

plot_embedding(X_tsne,
               "t-SNE embedding of the digits (time %.2fs)" %
               (time() - t0))

# TSNE для эмбеддингов

Теперь поиграем с словами.

Допустим у нас есть word2vec модель, обученная на российских новостях без предобработки. Давайте посмотри на то, какие эмбеддинги получились.

Для этого мы будем использовать TSNE.

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, LabelSet, HoverTool
import argparse
import numpy as np
from numpy.linalg import norm
from MulticoreTSNE import MulticoreTSNE as TSNE

In [ ]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In [ ]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [ ]:
ru_emb.vectors.shape

In [ ]:
top_k = 1000
words = ru_emb.index2word[:top_k]
embeddings = ru_emb.vectors[:top_k]
tsne = TSNE(n_jobs=4, perplexity=20)
Y = tsne.fit_transform(embeddings)
source = ColumnDataSource(data=dict(x=Y[:,0], y=Y[:,1], words=words))
hover = HoverTool(tooltips=[("word", "@words")])

TOOLS = ['pan', 'wheel_zoom', 'reset', hover]
p = figure(title='t-SNE embedding of top ' + str(len(Y)) + ' words',
        plot_width=1200, plot_height=720, tools=TOOLS)
p.scatter(x='x', y='y', size=8, source=source)
labels = LabelSet(x='x', y='y', text='words', level='glyph',
        x_offset=5, y_offset=5, source=source, render_mode='canvas')
p.add_layout(labels)
show(p)

# SVD

В этом задании вам предстоит работать с  датасетом, содержащем информацию о химическом составе рек


index_list - список id рек

feature_list - список признаков (они имеют вид год ПРОБЕЛ показатель)

data_matrix - собственно данные (строки соответствуют рекам из index_list, а столбцы - признакам из feature_list)

Ваше задание - сделать так, чтобы отрисовка всех объектов в координатах первых двух "скрытых" признаков, полученных с помощью SVD, давала хорошую визуализацию датасета. Изначально к этому будут некоторого рода препятствия; мы ожидаем, что вы их устраните.

In [ ]:
import pickle
index_list, feature_list, data_matrix = pickle.load(open('dataset_final', 'rb'))

In [ ]:
feature_list